In [ ]:
import pandas as pd
import numpy as np

In [ ]:
funds = pd.read_csv("/Users/akumaar/Downloads/13F_filtered.csv")
prices = pd.read_csv("/Users/akumaar/Downloads/prices_from_2013.csv")


In [ ]:
funds
len(np.unique(funds['iCIK']))
stock_pool = np.unique(funds['stock_id'])
stock_pool_real = pd.DataFrame(stock_pool, columns = ['stock_id'])
len(np.unique(funds['stock_id']))


In [ ]:
Fund_stocks = np.unique(funds['stock_id'])
len(np.unique(prices['stock_id']))

In [ ]:
#ETF Time Series
prices['date'] = pd.to_datetime(prices.date).dt.tz_localize(None)
prices.sort_values(by=['date'], inplace = True, ascending = True)

In [ ]:
XLF = prices.loc[prices['ticker'] == "XLF" ]
XLY = prices.loc[prices['ticker'] == "XLY" ]
XLK = prices.loc[prices['ticker'] == "XLK" ]
XLB = prices.loc[prices['ticker'] == "XLB" ]
XLI = prices.loc[prices['ticker'] == "XLI" ]
XLE = prices.loc[prices['ticker'] == "XLE" ]
XLU = prices.loc[prices['ticker'] == "XLU" ]
XLV = prices.loc[prices['ticker'] == "XLV" ]
XLP = prices.loc[prices['ticker'] == "XLP" ]
VOX = prices.loc[prices['ticker'] == "VOX" ]


In [ ]:
Etf_data = {'XLF':XLF['close'][:-1].values,
           'XLY':XLY['close'][:-1].values,
           'XLK':XLK['close'][:-1].values,
           'XLB':XLB['close'][:-1].values,
           'XLI':XLI['close'][:-1].values,
           'XLE':XLE['close'][:-1].values,
           'XLU':XLU['close'][:-1].values,
           'XLV':XLV['close'][:-1].values,
           'XLP':XLP['close'][:-1].values,
           'VOX':VOX['close'].values,
           'date':VOX['date'].values}

Etf_df = pd.DataFrame(Etf_data)

In [ ]:

x = prices.loc[prices['stock_id'] == "032511107" ]
#Etf_df['date'] = pd.to_datetime(Etf_df.date)
#len(np.unique(prices['stock_id']))
Etf_df

In [ ]:
joined = pd.merge(x, Etf_df, on = "date")
joined
joined[joined.columns[8:]].corr()['close'][1:]


In [ ]:
prices = prices[prices['stock_id'].isin(Fund_stocks)]
print(len(np.unique(prices['stock_id'])))

In [ ]:
#group prices data by unique stock id. 
#for every group, do the merge and correlation shit
#calculates each stocks industry through a correlation script
stock_group = prices.groupby(prices['stock_id'])
stock_group.get_group('000307108')
sector_categories = pd.DataFrame(columns = ['stock_id', 'sector'])
Headings = ["Financial", "Consumer Discretionary", "Technology", "Materials", "Industrial",
               "Energy", "Utilities", "Health Care", "Consumer Staples", "Communications"]

ETF_List = [XLF, XLY, XLK, XLB, XLI, XLE, XLU, XLV, XLP, VOX]

for i in stock_pool:
    print(i)
    try:
        #stock = stock_group.get_group(str(i))['close']
        stock = stock_group.get_group(str(i))
        print("after get group")
        #joined = pd.merge(x, Etf_df, on = "date")
        #sector = joined[joined.columns[8:]].corr()['close'][1:].idxmax()
        correlations = []
        for j in ETF_List:
            joined = pd.merge(stock, j, on = "date")
            correlation = np.corrcoef(joined['close_x'].values, joined['close_y'].values)
            correlations.append(correlation[1][0])
        sector_index = correlations.index(max(correlations))
        print(Headings[sector_index])
        sector_categories = sector_categories.append({'stock_id': i, 'sector':Headings[sector_index]}, ignore_index = True)
    except KeyError:
        sector_categories = sector_categories.append({'stock_id': i, 'sector':'NA'}, ignore_index = True)

In [ ]:

x = stock_group.get_group('000307108')
joined = pd.merge(x, VOX, on = "date")
joined
correlation = np.corrcoef(joined['close_x'].values, joined['close_y'].values)
correlation[0][1]

In [ ]:
sector_categories['sector'].value_counts()

In [ ]:
sector_categories.to_csv('/Users/akumaar/Downloads/sectors.csv')

In [ ]:
jj = pd.merge(funds, sector_categories, on = 'stock_id')


In [ ]:
##Generating Fund sectors by calculating stock sector holding proportions.
groupp = mergedFINAL.groupby('iCIK')
fund_sectors = pd.DataFrame(columns = ['fund', 'sector'])
for name,group in groupp:
    try:
        print(name)
        group = group[group.sector_x != 'NA']
        sector = group.groupby('sector_x')['iMARKET_VALUE'].sum().idxmax()    
        fund_sectors = fund_sectors.append({'fund': name, 'sector_x':sector}, ignore_index = True)
    except ValueError as e:
        print('value_error')

In [ ]:
fund_sectors = fund_sectors.drop(['sector'], axis = 1)

In [ ]:
fund_sectors.to_csv('/Users/akumaar/Downloads/fund_sectors_new.csv')

In [ ]:
mergedF = pd.read_csv('/Users/akumaar/Downloads/merged.csv')

In [ ]:
mergedFINAL = pd.merge(mergedF, funds, on = 'stock_id')

In [ ]:
mergedFINAL